In [66]:
import pandas as pd

In [67]:
# Read images and store them as Pandas DataFrames
# Header = None as the CSV's do not have any headers
img1 = pd.read_csv('img1.csv', header = None)
img2 = pd.read_csv('img2.csv', header = None)
img3 = pd.read_csv('img3.csv', header = None)
img4 = pd.read_csv('img4.csv', header = None)
img5 = pd.read_csv('img5.csv', header = None)


In [68]:
def pointCoord(img):
    points_xy = []
    sum = 0
    # Iterate over rows;  dataframe.shape() returns width, x counts x coordinate.
    for x, row in img.iterrows(): 
        # Iterate over columns.
        for y, value in row.items():
            # Check whether pixel at (x,y) is 1.
            # NOTE: df.at--> access via values, df.iat--> via int index
            if int(value) == 1:
                # If positive, store in list.
                # Add 1 because indexes start with 0
                points_xy.append(int(x) + 1)
                points_xy.append(int(y)+ 1)
                sum += 1
    print(points_xy)
    print(sum, "points found. \n")

In [71]:
#print(img1)
pointCoord(img1)
pointCoord(img2)
pointCoord(img3)
pointCoord(img4)
pointCoord(img5)

[1877, 1200]
1 points found. 

[1519, 1200]
1 points found. 

[1022, 1200]
1 points found. 

[519, 1200]
1 points found. 

[145, 1200]
1 points found. 



In [73]:
test1 = pd.read_csv("test1.csv", header = None)
print(test1)
pointCoord(test1)

   0  1  2  3  4  5  6  7  8
0  0  1  2  3  4  5  6  7  8
1  0  0  0  1  0  0  1  0  0
2  1  1  0  1  0  0  0  1  0
3  0  0  0  0  0  0  1  0  0
[1, 2, 2, 4, 2, 7, 3, 1, 3, 2, 3, 4, 3, 8, 4, 7]
8 points found. 

